In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
import functions
import scipy
import math
import operator
from functions import *
import itertools

In [2]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [3]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Columns: 1009 entries, Unnamed: 0 to nk_feat_387
dtypes: float64(1008), int64(1)
memory usage: 39.4 MB


In [4]:
selector = SelectKBest(score_func=f_classif, k=150)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [72] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


In [5]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
#gaussian process classification
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
gp = GaussianProcessClassifier(RBF(1.0))

In [9]:
gp.fit(x_train, np.squeeze(y_train))

GaussianProcessClassifier(kernel=RBF(length_scale=1))

In [10]:
#cross validation score
scores = cross_val_score(gp, x_train, np.ravel(y_train), cv=10, scoring='f1_micro', n_jobs=-1)
print(scores.mean())
print(scores.std())


KeyboardInterrupt: 

In [11]:
gp.fit(x_train, np.ravel(y_train))
y_test = gp.predict(x_test)
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
df_y_test = pd.DataFrame(data_y_test)
df_y_test.to_csv('outputs/output_gp.csv', index=False)

TypeError: _ArrayMemoryError.__init__() missing 1 required positional argument: 'dtype'